Multilabelled training on irmas dataset

In [ ]:
#Imports
import os
import random
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import soundfile as sf

from tqdm import tqdm
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
#Paths & Parameters
# Original IRMAS (single-label audio)
IRMAS_AUDIO_DIR = "/content/drive/MyDrive/irmas/IRMAS-TrainingData"

# New multilabel data
MIX_AUDIO_DIR = "/content/drive/MyDrive/irmas_multilabel_audio"
MEL_DIR = "/content/drive/MyDrive/irmas_multilabel_mels"

os.makedirs(MIX_AUDIO_DIR, exist_ok=True)
os.makedirs(MEL_DIR, exist_ok=True)

# Preprocessing parameters
TARGET_SR = 16000
TARGET_DURATION = 5.0
MIN_SILENCE_THRESH = 0.01
N_MELS = 128
HOP_LENGTH = 512
IMG_SIZE = 128

In [ ]:
#PREPROCESSING FUNCTION
def preprocess_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None, mono=True)

    # Resample
    if sr != TARGET_SR:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)
        sr = TARGET_SR

    # Peak normalization
    max_amp = np.max(np.abs(audio))
    if max_amp > 0:
        audio = audio / max_amp

    # Silence trimming
    non_silent = np.where(np.abs(audio) > MIN_SILENCE_THRESH)[0]
    if len(non_silent) > 0:
        audio = audio[non_silent[0]: non_silent[-1]]

    # Fix duration
    target_len = int(TARGET_SR * TARGET_DURATION)
    if len(audio) < target_len:
        audio = np.pad(audio, (0, target_len - len(audio)))
    else:
        audio = audio[:target_len]

    return audio, sr

In [ ]:
#MEL-SPECTROGRAM FUNCTION
def save_mel_spectrogram(audio, sr, save_path):
    mel = librosa.feature.melspectrogram(
        y=audio,
        sr=sr,
        n_mels=N_MELS,
        hop_length=HOP_LENGTH
    )

    mel_db = librosa.power_to_db(mel, ref=np.max)

    plt.figure(figsize=(3,3))
    plt.axis("off")
    librosa.display.specshow(mel_db, sr=sr, hop_length=HOP_LENGTH, cmap="magma")
    plt.savefig(save_path, bbox_inches="tight", pad_inches=0)
    plt.close()

In [ ]:
#CREATE MULTILABEL AUDIO
CLASS_FOLDERS = sorted([
    d for d in os.listdir(IRMAS_AUDIO_DIR)
    if os.path.isdir(os.path.join(IRMAS_AUDIO_DIR, d))
])
inst_to_idx = {inst: i for i, inst in enumerate(CLASS_FOLDERS)}
NUM_CLASSES = len(CLASS_FOLDERS)

labels = []
NUM_SAMPLES = 1000   # you can increase later

def mix_audios(file_paths, out_path):
    signals = []

    for f in file_paths:
        audio, _ = preprocess_audio(f)  # 🔥 reuse your preprocessing
        signals.append(audio)

    mixed = np.mean(signals, axis=0)
    sf.write(out_path, mixed, TARGET_SR)

for i in range(NUM_SAMPLES):
    chosen_classes = random.sample(CLASS_FOLDERS, random.randint(2, 3))
    chosen_files = []
    label_vec = np.zeros(NUM_CLASSES)

    for cls in chosen_classes:
        cls_dir = os.path.join(IRMAS_AUDIO_DIR, cls)
        file = random.choice(os.listdir(cls_dir))
        chosen_files.append(os.path.join(cls_dir, file))
        label_vec[inst_to_idx[cls]] = 1

    out_name = f"mix_{i}.wav"
    out_path = os.path.join(MIX_AUDIO_DIR, out_name)

    mix_audios(chosen_files, out_path)
    labels.append([out_name] + label_vec.tolist())

labels_df = pd.DataFrame(labels, columns=["file"] + CLASS_FOLDERS)
labels_df.to_csv("/content/drive/MyDrive/multilabel_labels.csv", index=False)

In [ ]:
#GENERATE MEL-SPECTROGRAMS FOR MULTILABEL AUDIO
for file in tqdm(labels_df["file"]):
    audio_path = os.path.join(MIX_AUDIO_DIR, file)
    img_path = os.path.join(MEL_DIR, file.replace(".wav", ".png"))

    audio, sr = preprocess_audio(audio_path)
    save_mel_spectrogram(audio, sr, img_path)

100%|██████████| 1000/1000 [02:28<00:00,  6.73it/s]


In [ ]:
#LOAD DATA FOR TRAINING
X, y = [], []

for _, row in labels_df.iterrows():
    img_path = os.path.join(MEL_DIR, row["file"].replace(".wav", ".png"))
    if not os.path.exists(img_path):
        continue

    # Load image and normalize
    img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img = img_to_array(img) / 255.0
    X.append(img)

    # Convert label to float
    label_vec = row[1:].values.astype("float32")
    y.append(label_vec)

# Convert lists to NumPy arrays with proper dtype
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

print("X shape:", X.shape)
print("y shape:", y.shape)
print("X dtype:", X.dtype)
print("y dtype:", y.dtype)

# ========================
# TRAIN/VALIDATION SPLIT
# ========================
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val:", X_val.shape, "y_val:", y_val.shape)

X shape: (1000, 128, 128, 3)
y shape: (1000, 11)
X dtype: float32
y dtype: float32
X_train: (800, 128, 128, 3) y_train: (800, 11)
X_val: (200, 128, 128, 3) y_val: (200, 11)


In [ ]:
#MULTILABEL CNN MODEL
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.5),

    Dense(NUM_CLASSES, activation="sigmoid")  # MULTILABEL
])

model.compile(
    optimizer=Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["binary_accuracy", Precision(), Recall()]
)

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 11)             │         2,827 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,518,859 (24.87 MB)

 Trainable params: 6,518,859 (24.87 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# TRAIN THE MODEL
# ========================
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32
)

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - binary_accuracy: 0.6769 - loss: 0.6184 - precision_2: 0.2294 - recall_2: 0.1736 - val_binary_accuracy: 0.7764 - val_loss: 0.5406 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - binary_accuracy: 0.7632 - loss: 0.5619 - precision_2: 0.2042 - recall_2: 0.0125 - val_binary_accuracy: 0.7764 - val_loss: 0.5326 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - binary_accuracy: 0.7696 - loss: 0.5561 - precision_2: 0.2099 - recall_2: 0.0069 - val_binary_accuracy: 0.7764 - val_loss: 0.5279 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - binary_accuracy: 0.7685 - loss: 0.5487 - precision_2: 0.3676 - recall_2: 0.0157 - val_binary_accuracy: 0.7764 - val_loss: 0.5256 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/ste

In [ ]:
#SAVE MODEL
model.save("/content/drive/MyDrive/irmas_multilabel_cnn.keras")